In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd


In [3]:
tripss = pd.read_csv('C:/Users/kellw/Downloads/trips.csv',sep=',', names = ["trip_id", "rideable_type","started_at","ended_at","start_station_id","end_station_id","rider_id"])

In [4]:
display(tripss)

,trip_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,rider_id
0,89E7AA6C29227EFF,classic_bike,2021-02-12 16:14:56,2021-02-12 16:21:43,525,660,71934
1,0FEFDE2603568365,classic_bike,2021-02-14 17:52:38,2021-02-14 18:12:09,525,16806,47854
2,E6159D746B2DBB91,electric_bike,2021-02-09 19:10:18,2021-02-09 19:19:10,KA1503000012,TA1305000029,70870
3,B32D3199F1C2E75B,classic_bike,2021-02-02 17:49:41,2021-02-02 17:54:06,637,TA1305000034,58974
4,83E463F23575F4BF,electric_bike,2021-02-23 15:07:23,2021-02-23 15:22:37,13216,TA1309000055,39608
...,...,...,...,...,...,...,...
10144,23E3E96EA5CED1EB,classic_bike,2021-02-26 00:30:57,2021-02-26 00:37:34,13266,15650,16641
10145,53D507A060C1936C,classic_bike,2021-02-24 21:20:49,2021-02-24 21:25:27,13292,15650,24289
10146,9EE00DE7D938384D,classic_bike,2021-02-27 16:20:23,2021-02-27 16:24:08,13292,15650,5434
10147,6C5C404A692E16B6,classic_bike,2021-02-25 14:20:37,2021-02-25 14:43:54,13257,15650,56858


In [5]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [11]:
times = pd.date_range(start='2021-02-01', end='2022-02-28', freq = 'D')
print(times)
print(times.dtype)




DatetimeIndex(['2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04',
               '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08',
               '2021-02-09', '2021-02-10',
               ...
               '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22',
               '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26',
               '2022-02-27', '2022-02-28'],
              dtype='datetime64[ns]', length=393, freq='D')
datetime64[ns]


In [9]:
minstartdate = tripss['started_at'].min()

In [10]:
maxstartdate = tripss['started_at'].max()
print(minstartdate)
print(maxstartdate)

2021-02-01 01:56:22
2021-02-28 23:59:41


In [12]:
display(times)

DatetimeIndex(['2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04',
               '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08',
               '2021-02-09', '2021-02-10',
               ...
               '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22',
               '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26',
               '2022-02-27', '2022-02-28'],
              dtype='datetime64[ns]', length=393, freq='D')

In [14]:
dimtime = pd.DataFrame(times)
headers = ["adate"]
dimtime.columns = headers



In [21]:
display(dimtime)

,New_ID,adate,ayear,amonth,aquarter,adayofweek,adayofmonth
0,1,2021-02-01,2021,2,1,0,1
1,2,2021-02-02,2021,2,1,1,2
2,3,2021-02-03,2021,2,1,2,3
3,4,2021-02-04,2021,2,1,3,4
4,5,2021-02-05,2021,2,1,4,5
...,...,...,...,...,...,...,...
388,389,2022-02-24,2022,2,1,3,24
389,390,2022-02-25,2022,2,1,4,25
390,391,2022-02-26,2022,2,1,5,26
391,392,2022-02-27,2022,2,1,6,27


In [16]:
dimtime['ayear'] = pd.DatetimeIndex(dimtime['adate']).year
dimtime['amonth'] = pd.DatetimeIndex(dimtime['adate']).month
dimtime['aquarter'] = pd.DatetimeIndex(dimtime['adate']).quarter
dimtime['adayofweek'] = pd.DatetimeIndex(dimtime['adate']).dayofweek
dimtime['adayofmonth'] = pd.DatetimeIndex(dimtime['adate']).day

dimtime.head()

,adate,ayear,amonth,aquarter,adayofweek,adayofmonth
0,2021-02-01,2021,2,1,0,1
1,2021-02-02,2021,2,1,1,2
2,2021-02-03,2021,2,1,2,3
3,2021-02-04,2021,2,1,3,4
4,2021-02-05,2021,2,1,4,5


In [ ]:
df.insert(0, 'New_ID', range(880, 880 + len(df)))
df = df.reset_index()
df = df.rename(columns={"index":"New_ID"})

dimtime.insert(0, 'New_ID',range(1, 1+len(dimtime)))

In [18]:
dimtime.insert(0, 'time_id',range(1, 1+len(dimtime)))


In [20]:
dimtime.rename(columns={'New_ID': 'time_id'})

,time_id,adate,ayear,amonth,aquarter,adayofweek,adayofmonth
0,1,2021-02-01,2021,2,1,0,1
1,2,2021-02-02,2021,2,1,1,2
2,3,2021-02-03,2021,2,1,2,3
3,4,2021-02-04,2021,2,1,3,4
4,5,2021-02-05,2021,2,1,4,5
...,...,...,...,...,...,...,...
388,389,2022-02-24,2022,2,1,3,24
389,390,2022-02-25,2022,2,1,4,25
390,391,2022-02-26,2022,2,1,5,26
391,392,2022-02-27,2022,2,1,6,27


In [22]:
minstartdate1 = '2021-02-01'

maxstartdate1 = '2022-02-01'

times1 = pd.date_range(start=minstartdate1, end=maxstartdate1, freq = 'D')
print(times)

DatetimeIndex(['2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04',
               '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08',
               '2021-02-09', '2021-02-10',
               ...
               '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22',
               '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26',
               '2022-02-27', '2022-02-28'],
              dtype='datetime64[ns]', length=393, freq='D')


In [29]:
times3 = pd.date_range(start=minstartdate, end=maxstartdate, freq = 'D')
print(times3)

DatetimeIndex(['2021-02-01 01:56:22', '2021-02-02 01:56:22',
               '2021-02-03 01:56:22', '2021-02-04 01:56:22',
               '2021-02-05 01:56:22', '2021-02-06 01:56:22',
               '2021-02-07 01:56:22', '2021-02-08 01:56:22',
               '2021-02-09 01:56:22', '2021-02-10 01:56:22',
               '2021-02-11 01:56:22', '2021-02-12 01:56:22',
               '2021-02-13 01:56:22', '2021-02-14 01:56:22',
               '2021-02-15 01:56:22', '2021-02-16 01:56:22',
               '2021-02-17 01:56:22', '2021-02-18 01:56:22',
               '2021-02-19 01:56:22', '2021-02-20 01:56:22',
               '2021-02-21 01:56:22', '2021-02-22 01:56:22',
               '2021-02-23 01:56:22', '2021-02-24 01:56:22',
               '2021-02-25 01:56:22', '2021-02-26 01:56:22',
               '2021-02-27 01:56:22', '2021-02-28 01:56:22'],
              dtype='datetime64[ns]', freq='D')


In [30]:
minstartdate4 = tripss['started_at'].min()
minstartdate4 = minstartdate4.str(:10)
maxstartdate4 = tripss['started_at'].max()
maxstartdate4 = maxstartdate4.str(:10)
print(minstartdate4)
print(maxstartdate4)

SyntaxError: invalid syntax (1632646877.py, line 2)

In [32]:
dimtime3 = pd.DataFrame(times3)

In [35]:
display(dimtime3)

,time_id
0,2021-02-01 01:56:22
1,2021-02-02 01:56:22
2,2021-02-03 01:56:22
3,2021-02-04 01:56:22
4,2021-02-05 01:56:22
5,2021-02-06 01:56:22
6,2021-02-07 01:56:22
7,2021-02-08 01:56:22
8,2021-02-09 01:56:22
9,2021-02-10 01:56:22


In [34]:
headers = ["time_id"]
dimtime3.columns = headers

In [36]:
dimtime3['adate'] = pd.DatetimeIndex(dimtime3['time_id']).date
dimtime3['ayear'] = pd.DatetimeIndex(dimtime3['time_id']).year
dimtime3['amonth'] = pd.DatetimeIndex(dimtime3['time_id']).month
dimtime3['aquarter'] = pd.DatetimeIndex(dimtime3['time_id']).quarter
dimtime3['adayofweek'] = pd.DatetimeIndex(dimtime3['time_id']).dayofweek
dimtime3['adayofmonth'] = pd.DatetimeIndex(dimtime3['time_id']).day

dimtime3.head()

,time_id,adate,ayear,amonth,aquarter,adayofweek,adayofmonth
0,2021-02-01 01:56:22,2021-02-01,2021,2,1,0,1
1,2021-02-02 01:56:22,2021-02-02,2021,2,1,1,2
2,2021-02-03 01:56:22,2021-02-03,2021,2,1,2,3
3,2021-02-04 01:56:22,2021-02-04,2021,2,1,3,4
4,2021-02-05 01:56:22,2021-02-05,2021,2,1,4,5


In [ ]:
dimtime1.columns = headers
dimtime['adate'] = pd.DatetimeIndex(dimtime['time_id']).date
dimtime1 = dimtime1.drop(columns = ['time_id'])
dimtime1.insert(0, 'time_id',range(1, 1+len(dimtime)))
display(dimtime1)